In [23]:
import numpy as np
# from graphviz import Digraph


class Matrix:

    def __init__(self, val, _op='', _desc=()):
        self.val = val
        self.shape = val.shape
        self.grad = np.zeros(self.shape)
        self._backprop = lambda: None
        self._prev = set(_desc)
        self._op = _op

    def __add__(self, other):
        other = other if isinstance(other, Matrix) else Matrix(other)
        res = Matrix(self.val + other.val, '+', (self, other))

        def _backprop():
            if self.shape == res.shape:
                self.grad += res.grad
            elif self.shape[0] == res.shape[0]:
                self.grad += res.grad.sum(1, keepdims=True)
            else:
                self.grad += res.grad.sum()
            # print("ADD Self:", self.grad)
            if other.shape == res.shape:
                other.grad += res.grad
            elif other.shape[0] == res.shape[0]:
                other.grad += res.grad.sum(1, keepdims=True)
            else:
                other.grad += res.grad.sum()
            # print("ADD -other:", other.grad)

        res._backprop = _backprop
        return res

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Matrix) else Matrix(other)
        res = Matrix(self.val * other.val, '*', (self, other))

        def _backprop():
            if self.shape == res.shape:
                self.grad += other.val * res.grad
            elif self.shape[0] == res.shape[0]:
                self.grad += (other.val * res.grad).sum(1, keepdims=True)
            else:
                self.grad += (other.val * res.grad).sum()
            # print("Multiply Self:", self.grad)

            if other.shape == res.shape:
                other.grad += self.val * res.grad
            elif other.shape[0] == res.shape[0]:
                other.grad += (self.val * res.grad).sum(1, keepdims=True)
            else:
                other.grad += (self.val * res.grad).sum()
            # print("Multiply -other:", other.grad)

        res._backprop = _backprop
        return res

    def __rmul__(self, other):
        return self * other

    def __neg__(self):
        return self * np.array([-1])

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __matmul__(self, other):
        res = Matrix(self.val @ other.val, '@', (self, other))

        def _backprop():
            self.grad += res.grad @ other.val.T
            other.grad += self.val.T @ res.grad

        res._backprop = _backprop
        return res

    def __pow__(self, power):
        res = Matrix(self.val ** power, f'**{power}', (self,))

        def _backprop():
            self.grad += res.grad * (power * (self.val ** (power - 1)))
            # print("Power backprop: ", self.grad)

        res._backprop = _backprop
        return res

    def __truediv__(self, other):
        return self * (other ** -1)

    def __rtruediv__(self, other):
        return other * (self ** -1)

    def T(self):
        res = Matrix(self.val.T, 'T', (self,))

        def _backprop():
            self.grad += res.grad.T

        res._backprop = _backprop
        return res

    def log(self):
        res = Matrix(np.log(self.val), _op='log', _desc=(self,))

        def _backprop():
            self.grad += res.grad * (self.val ** -1)

        res._backprop = _backprop
        return res

    def softmax(self):
        # print("softmax:", self.shape)
        shift = self.val - np.max(self.val, axis=1, keepdims=True)
        exp_shift = np.exp(shift)
        smax = exp_shift / np.sum(exp_shift, axis=1, keepdims=True)
        res = Matrix(smax, 'smax', (self,))

        def _backprop():
            for i in range(smax.shape[0]):
                grad_matrix = -np.outer(smax[i], smax[i]) + np.diag(smax[i])
                self.grad[i] += (res.grad[i].reshape(1, -1) * grad_matrix).sum(1)

        res._backprop = _backprop
        return res

    def cross_entropy(self, gold):
        assert self.shape[0] == gold.shape[0], f"number of outputs must be equal: {self.shape[0]} and {gold.shape[0]}"
        argmax_gold = np.argmax(gold.val, axis=1)
        ceLoss = Matrix(np.array((-1.0 / self.shape[0]) * np.sum(np.log(self[np.arange(self.shape[0]), argmax_gold]))),
                        _op='cen',
                        _desc=(self, gold))

        def _backprop():
            self.grad += -1.0 * (gold.val / self.val) * ceLoss.grad
            # no need to backprop for gold values

        ceLoss._backprop = _backprop
        return ceLoss

    def __getitem__(self, index):
        return self.val[index]

    def backprop(self):
        order = []
        visited = set()

        def build_dependency_tree(node):
            if node not in visited:
                visited.add(node)
                for prev_node in node._prev:
                    build_dependency_tree(prev_node)
                order.append(node)
                # print("Step=", v._op)

        build_dependency_tree(self)

        self.grad = np.ones(self.shape)
        for v in reversed(order):
            # print(v.val, "Operator=", v._op, v._backprop)
            v._backprop()
#             print("Grad:", v.grad.sum()) 
        return order


def zero_grad(order):
    for v in order:
        v.grad = np.zeros(v.grad.shape)

In [11]:
import nltk
nltk.download('punkt')
nltk.download("gutenberg") 
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import numpy as np
import pandas as pd

[nltk_data] Downloading package punkt to /home1/tejomay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     /home1/tejomay/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home1/tejomay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home1/tejomay/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home1/tejomay/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
def preprocess_corpus(data):

  stop_words = set(stopwords.words('english'))   
  punctuation = {'.','!', "'", "''", '(', ')', ',', '.', ':', ';', '?', '[', ']', '``', ' ', '_', '"','*','%','$','&','+','-','/','\\','`','<','>','=','@'}

  cleaned_data = []
  unique_words = []
  length = 0

  lemmatizer = WordNetLemmatizer()

  for sent in data:
    new_sent = []

    for word in sent:
      # removing any punctuations appended with the word
      for w in word: 
          if(w in punctuation or w.isnumeric()):
            word = word.replace(w,'')

      # lowercasing the word
      word = word.lower()
      # removing stopwords and digits
      if(word not in stop_words and word.isnumeric() == False and word !=''): 
          # lemmatize
#           word = lemmatizer.lemmatize(word)  
          # adding cleaned words to the sentence
          new_sent.append(word)
          # checking for unique words
          if(word not in unique_words):
            unique_words.append(word)

    # adding valid sentences to the data
    if(len(new_sent) > 1):
#         final_sent = []
#         for word in new_sent:
#             if word not in final_sent:
#                 final_sent.append(word)
        cleaned_data.append(new_sent)
        length += len(new_sent)

  print("Average length of sentences : ", length/(len(cleaned_data)))

  return cleaned_data, unique_words

  
def create_vocab(words):
    sorted_words = sorted(words) 
    vocab = {word: sorted_words.index(word) for word in sorted_words}
    vocab['.'] = len(sorted_words)
    return sorted_words, vocab


def word_to_onehot(word, vocab):
  onehot = np.zeros((len(vocab),))
  # print(word)
  onehot[vocab[word]] = 1.0
  return onehot

In [13]:
def create_data(sentences, vocab, window_size=1):
    created_data = []
    # for i in range(len(sentences)):
    #     context_and_word = []
    #     for j in range(i-window_size, i+window_size+1):
    #         if j > 0 and j < len(sentences) and j != i:
    #             context_and_word.append(word_to_onehot(sentences[i][j], vocab))
    #     context_and_word.append(word_to_onehot(sentences[i], vocab))
    #     data.append(context_and_word)
    count = 0
    for sentence in sentences:
        sentence = ['.']*window_size + sentence + ['.']*window_size 
#         print(sentence)
        for i in range(window_size, len(sentence) - window_size):
            context_and_word = []
            for j in range(i-window_size, i+window_size+1):
                if j != i:
                    context_and_word.append(sentence[j])
            context_and_word.append(sentence[i])
            created_data.append(context_and_word)
        count += 1
        if count%5000 == 0:
            print(f"{count} sentences processed.")

    return created_data

In [14]:
sentences = nltk.corpus.gutenberg.sents()
# print("No. of sentences: ",len(sentences))
# print("A sample sentence: ",sentences[0])
data, unique_words = preprocess_corpus(sentences)
print("No. of samples: ",len(data))
print("A sample sentence: ", data[0]) 
print("No. of unique words: ", len(unique_words))
sorted_words, vocab = create_vocab(unique_words)

Average length of sentences :  11.38591095652952
No. of samples:  89417
A sample sentence:  ['emma', 'jane', 'austen']
No. of unique words:  41361


In [15]:
final_data = create_data(data, vocab, window_size=1)

5000 sentences processed.
10000 sentences processed.
15000 sentences processed.
20000 sentences processed.
25000 sentences processed.
30000 sentences processed.
35000 sentences processed.
40000 sentences processed.
45000 sentences processed.
50000 sentences processed.
55000 sentences processed.
60000 sentences processed.
65000 sentences processed.
70000 sentences processed.
75000 sentences processed.
80000 sentences processed.
85000 sentences processed.


In [16]:
len(final_data), len(final_data[0])

(1018094, 3)

In [20]:
import time

def forward_pass(context, label_word, context_weight, softmax_weight, d, method="cbow"):  # data in the form of list of context words
    # context word: batch_size x |V|   weights: |V| x d
    net = (sum(context) / Matrix(np.array([len(context)]).reshape(1,1))) @ context_weight
    out = net @ softmax_weight
    smax = out.softmax()
    loss = smax.cross_entropy(label_word)
    return loss


# returns context and label
def convert_to_onehot(batch):
    context = []
    label_word = np.zeros((len(batch), len(vocab)))
    for i in range(len(batch[0])-1):
        vectors = np.zeros((len(batch), len(vocab)))
        for j in range(len(batch)):
            onehot = word_to_onehot(batch[j][i], vocab)
            vectors[j,:] = onehot
        vectors = Matrix(vectors)
        context.append(vectors)
    for j in range(len(batch)):
        onehot = word_to_onehot(batch[j][-1], vocab)
        label_word[j,:] = onehot
    label_word = Matrix(label_word)
    
    return context, label_word


# data shape: N x C x |V|
# data has first C-1 elements as context and last element as label word
def train(data, weights, batch_size=4, num_epochs=100, lr=0.01, dim=100, method="cbow"):
    # context_weights = Matrix(np.random.randn(data.shape[2], dim))
    # softmax_weights = Matrix(np.random.randn(dim, data.shape[2]))
    
    for epoch in range(num_epochs):
        step = 0
        losses = []
        tolerance = 0
        final_weights = [weights[0].val, weights[1].val]
        time_i = time.time()
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size-1]
            context, label_word = convert_to_onehot(batch)
            loss = forward_pass(context, label_word, weights[0], weights[1], dim)
            order = loss.backprop()
#             print("Weight:", weights[0].shape, weights[0].grad.shape)
            for weight in weights:
                weight.val -= lr * weight.grad
#             print("Weight:", weights[0].shape)
            zero_grad(order)
            step += 1
            if step % 100 == 0:
                time_j = time.time()
                print(f"Epoch: {epoch+1} | Step: {step} | Loss: {loss.val:.5f} | Time: {time_j - time_i}")
        losses.append(loss.val)
        if len(losses) > 10:
            if losses[epoch] >= losses[epoch-1]:
                tolerance += 1
            else:
                final_weights = [weights[0].val, weights[1].val]
                tolerance = 0
        if tolerance > 4:
            break
            
    return final_weights

In [21]:
context, label_word = convert_to_onehot(final_data[:4])
len(context), context[0].shape, label_word.shape

(2, (4, 41362), (4, 41362))

In [24]:
# from backpropagation import Matrix, zero_grad
dim = 100
context_weight = Matrix(np.random.randn(len(vocab), dim))
softmax_weight = Matrix(np.random.randn(dim, len(vocab)))
weights = [context_weight, softmax_weight]
weights = train(final_data, weights, batch_size=4, num_epochs=30, lr=0.01, dim=dim)

AttributeError: 'int' object has no attribute 'shape'

In [31]:
final_data[:4]

[['.', 'jane', 'emma'],
 ['emma', 'austen', 'jane'],
 ['jane', '.', 'austen'],
 ['.', 'woodhouse', 'emma']]